## Imports

In [1]:
import numpy as np
import pandas as pd
import torch
from transformers import set_seed

In [2]:
from tsfm_public import TimeSeriesPreprocessor
from tsfm_public.models.tspulse import TSPulseForReconstruction

## Preparing the Dataset

In [3]:
# Set seed for reproducibility
SEED = 42
set_seed(SEED)
CONTEXT_LENGTH = 512
PREDICTION_LENGTH = 0

TARGET_DATASET = "etth1"
dataset_path = "https://raw.githubusercontent.com/zhouhaoyi/ETDataset/main/ETT-small/ETTh1.csv"

In [4]:
timestamp_column = "date"
id_columns = []  # mention the ids that uniquely identify a time-series.
target_columns = ["HUFL", "HULL", "MUFL", "MULL", "LUFL", "LULL", "OT"]

data = pd.read_csv(
    dataset_path,
    parse_dates=[timestamp_column],
)

print(data.shape)
print(data.head())

column_specifiers = {
    "timestamp_column": timestamp_column,
    "id_columns": id_columns,
    "target_columns": target_columns,
    "control_columns": [],
}

(17420, 8)
                 date   HUFL   HULL   MUFL   MULL   LUFL   LULL         OT
0 2016-07-01 00:00:00  5.827  2.009  1.599  0.462  4.203  1.340  30.531000
1 2016-07-01 01:00:00  5.693  2.076  1.492  0.426  4.142  1.371  27.787001
2 2016-07-01 02:00:00  5.157  1.741  1.279  0.355  3.777  1.218  27.787001
3 2016-07-01 03:00:00  5.090  1.942  1.279  0.391  3.807  1.279  25.044001
4 2016-07-01 04:00:00  5.358  1.942  1.492  0.462  3.868  1.279  21.948000


In [5]:
# introducing NaN values to mimic real world missing values

nan_fraction = 0.2
num_values = data.shape[0] * len(target_columns)
num_nans = int(nan_fraction * num_values)

rows = np.random.randint(0, data.shape[0], size=num_nans)
cols = np.random.randint(1, len(target_columns), size=num_nans)

for r, col in zip(rows, cols):
    data.iat[r, col] = np.nan

data.head(100)

,date,HUFL,HULL,MUFL,MULL,LUFL,LULL,OT
0,2016-07-01 00:00:00,5.827,2.009,1.599,0.462,4.203,1.340,30.531000
1,2016-07-01 01:00:00,5.693,NaN,1.492,0.426,4.142,1.371,27.787001
2,2016-07-01 02:00:00,5.157,1.741,1.279,0.355,NaN,1.218,27.787001
3,2016-07-01 03:00:00,NaN,1.942,NaN,0.391,3.807,1.279,25.044001
4,2016-07-01 04:00:00,5.358,1.942,1.492,0.462,3.868,1.279,21.948000
...,...,...,...,...,...,...,...,...
95,2016-07-04 23:00:00,NaN,6.497,9.346,3.482,3.168,1.980,25.466000
96,2016-07-05 00:00:00,11.989,5.626,8.777,2.949,3.198,1.980,25.958000
97,2016-07-05 01:00:00,12.525,6.296,8.955,NaN,NaN,2.010,25.958000
98,2016-07-05 02:00:00,12.324,6.296,8.813,3.376,2.985,1.919,26.028000


In [6]:
tsp = TimeSeriesPreprocessor(
    **column_specifiers,
    context_length=CONTEXT_LENGTH,
    prediction_length=PREDICTION_LENGTH,
    scaling=True,
    encode_categorical=False,
    scaler_type="standard",
)

## Getting the pre-trained TSPulse Model

In [7]:
model_path = (
    "/dccstor/dnn_forecasting/subodh/tspulse_models/tspulse_hybrid_imp/tspulse_model"
)
model = TSPulseForReconstruction.from_pretrained(
    model_path, num_input_channels=tsp.num_input_channels, mask_type="user"
)

device = "cuda" if torch.cuda.is_available() else "cpu"
model = model.to(device).float()

## Using Imputation Pipeline for Zero-Shot Imputation

In [8]:
from tsfm_public.toolkit.time_series_imputation_pipeline import TimeSeriesImputationPipeline

# tsp.train(data)   # train the tsp 
pipe = TimeSeriesImputationPipeline(model, feature_extractor=tsp, device=device)

Device set to use cuda


In [9]:
out = pipe(data)

In [10]:
out.shape

(17420, 15)

In [11]:
out.head()

,date,HUFL,HULL,MUFL,MULL,LUFL,LULL,OT,HUFL_imputed,HULL_imputed,MUFL_imputed,MULL_imputed,LUFL_imputed,LULL_imputed,OT_imputed
0,2016-07-01 00:00:00,5.827,2.009,1.599,0.462,4.203,1.340,30.531000,5.827000,2.009000,1.599000,0.462,4.203000,1.340,30.531000
1,2016-07-01 01:00:00,5.693,NaN,1.492,0.426,4.142,1.371,27.787001,5.693000,2.404703,1.492000,0.426,4.142000,1.371,27.787001
2,2016-07-01 02:00:00,5.157,1.741,1.279,0.355,NaN,1.218,27.787001,5.157000,1.741000,1.279000,0.355,3.852766,1.218,27.787001
3,2016-07-01 03:00:00,NaN,1.942,NaN,0.391,3.807,1.279,25.044001,5.467124,1.942000,1.772878,0.391,3.807000,1.279,25.044001
4,2016-07-01 04:00:00,5.358,1.942,1.492,0.462,3.868,1.279,21.948000,5.358000,1.942000,1.492000,0.462,3.868000,1.279,21.948000
